In [45]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [46]:
query = """query swaps{
  swaps(orderBy: timestamp, orderDirection: desc, where:{pair: "0x000bc4f31d2c8dcc66f6f1c93bb00920e3ea6c5d"}) {
    Pool_id:id
    timestamp
    amount0In
    amount1In
    amount0Out
    amount1Out
  
    pair {
      token0 {
        id
        symbol
        tradeVolume
        liquidity_pool:totalLiquidity
      }
      token1 {
        id
        symbol
        tradeVolume
        liquidity_pool:totalLiquidity
      }
    volumeUSD
    reserveUSD
    }
  }
}"""

In [ ]:
url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2'
r = requests.post(url, json={'query': query})
print(r.status_code)
print(r.text)

In [ ]:
json_data = json.loads(r.text)

In [ ]:
json_data

In [ ]:
def transaction_type(row):
    if row['amount0In'] == '0':
        return 'sell'
    else:
        return 'buy'
def pairRetrieve(row,key,column):
    print(row)
    return row[column][key]

In [ ]:
df_data = json_data['data']['swaps']
df = pd.DataFrame(df_data)
keys = df['pair'][0].keys()
for i in keys:
    df[i] = df.apply(pairRetrieve,args=(i,'pair'),axis =1)

In [ ]:
df['side'] = df.apply(transaction_type,axis=1)

In [ ]:
keys = df['token0'][0].keys()
for i in keys:
    df[i + '_token0'] = df.apply(pairRetrieve,args=(i,'token0'),axis =1)
    
keys = df['token1'][0].keys()
for i in keys:
    df[i + '_token1'] = df.apply(pairRetrieve,args=(i,'token1'),axis =1)

In [ ]:

df=df.drop(['pair'], axis = 1)
df=df.drop(['token0'], axis = 1)
df=df.drop(['token1'], axis = 1)


In [ ]:
query2 = """{
 pairs(first: 100, orderBy: reserveUSD, orderDirection: desc) {
   id
  reserveUSD
  token0{
    symbol}
  token1{
    symbol
  }
  }
 }"""

In [ ]:
url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2'
r = requests.post(url, json={'query': query2})
print(r.status_code)
print(r.text)
json_data = json.loads(r.text)

In [ ]:
df2 = json_data['data']['pairs']
df2 = pd.DataFrame(df2)

In [ ]:
df3 = pd.DataFrame(df2[0:10])

In [ ]:
def func(row):
    return row['token0']['symbol'] + ' ' + row['token1']['symbol']

In [ ]:
df3['token'] = df3.apply(func,axis=1)

In [ ]:
mylabels = list(df3['token'])
values = np.array(df3['reserveUSD'])


plt.pie(values, labels = mylabels)
plt.show() 

In [ ]:
sum_total = df2.astype({'reserveUSD':'float64'}).sum()
sum_top10 = df3.astype({'reserveUSD':'float64'}).sum()

In [ ]:
mylabels = ['Top100','Top10']
values =[float(sum_total['reserveUSD']), float(sum_top10['reserveUSD'])]

plt.pie(values, labels = mylabels)
plt.show() 